<a href="https://colab.research.google.com/github/falken20/Colaboratory-projects/blob/master/Python_Detector_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Detection example using mediapipe**

#### **Resources**:

Web: [mediapipe](https://mediapipe-studio.webapps.google.com/home)

Model: [efficientdet_lite0.tflite](https://github.com/schu-lab/Tensorflow-Object-Detection/blob/main/efficientdet_lite0.tflite)

In [ ]:
!pip install mediapipe

In [ ]:
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

In [ ]:
# The mediapipe object detector requires a trained model
# More information and trained models:
# https://developers.google.com/mediapipe/solutions/vision/object_detector/index#models
MODEL_PATH = './trained_model/efficientdet_lite0.tflite'
IMAGE_FILE = "https://storage.googleapis.com/mediapipe-tasks/object_detector/cat_and_dog.jpg"

In [ ]:
MARGIN = 10  # pixels
ROW_SIZE = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
TEXT_COLOR = (255, 0, 0)  # red


def visualize(
    image,
    detection_result
) -> np.ndarray:
    """Draws bounding boxes on the input image and return it.
    Args:
      image: The input RGB image.
      detection_result: The list of all "Detection" entities to be visualize.
    Returns:
      Image with bounding boxes.
    """
    for detection in detection_result.detections:
        # Draw bounding_box
        bbox = detection.bounding_box
        start_point = bbox.origin_x, bbox.origin_y
        end_point = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
        cv2.rectangle(image, start_point, end_point, TEXT_COLOR, 3)

        # Draw label and score
        category = detection.categories[0]
        category_name = category.category_name
        probability = round(category.score, 2)
        result_text = category_name + ' (' + str(probability) + ')'
        text_location = (MARGIN + bbox.origin_x,
                         MARGIN + ROW_SIZE + bbox.origin_y)
        cv2.putText(image, result_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                    FONT_SIZE, TEXT_COLOR, FONT_THICKNESS)

    return image

In [ ]:
def detect_image() -> None:

    options = vision.ObjectDetectorOptions(
        base_options=python.BaseOptions(model_asset_path=MODEL_PATH),
        max_results=5,
        running_mode=vision.RunningMode.IMAGE,
        score_threshold=0.5
    )

    detector = vision.ObjectDetector.create_from_options(options)

    # Load the input image
    image = cv2.imread(IMAGE_FILE)
    cv2.imshow(image)

    # Detect objects in the input image
    detection_result = detector.detect(image)

    # Process the detection result, in this case, visualize it
    image_copy = np.copy(image.numpy_view())
    annotated_image = visualize(image_copy, detection_result)
    rgb_annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    cv2.imshow(rgb_annotated_image)

In [ ]:
detect_image("any_video.avi")

### **Human Detection example using cv2**

#### **Resources**:

Classifier: [haarcascade_fullbody.xml](https://github.com/codingforentrepreneurs/OpenCV-Python-Series/tree/master/src/cascades/data)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
cap = cv2.VideoCapture('in.avi')
human_cascade = cv2.CascadeClassifier('haarcascade_fullbody.xml')

In [ ]:
while (True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    humans = human_cascade.detectMultiScale(gray, 1.9, 1)

    # Display the resulting frame
    for (x, y, w, h) in humans:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # cv2.imshow('frame', frame)
    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

### **Object Detection example using YOLO**

#### **Resources**:

Model: [yolo.h5](https://github.com/OlafenwaMoses/ImageAI/releases/tag/1.0/)

In [ ]:
!pip install imageAI
!pip install opencv-python
!pip install Pillow

In [4]:
# Creating an object of the object detection class
from imageai.Detection import ObjectDetection

obj_detect = ObjectDetection()

In [ ]:
# Setting and loading the YOLO model
obj_detect.setModelTypeAsYOLOv3()

obj_detect.setModelPath("https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo.h5")
obj_detect.loadModel()

In [ ]:
# Detecting objects
detected_obj = obj_detect.detectObjectsFromImage(
    input_image="any_image.jpg",
    output_image_path="./output_image.jpg"
)

In [ ]:
# Displaying information about detected objects
for obj in detected_obj:
  print(obj["name"] + "-" + str(obj["percentage_probability"]),
        obj["box_points"])
  
# Output example
# bicycle-97.5232720375061 [345, 60, 654, 238]
# car-99.8550295829773 [7, 250, 778, 519]
# person-99.85941052436829 [682, 214, 783, 476]
# person-99.91627335548401 [76, 218, 214, 502]
# person-68.58709454536438 [279, 276, 348, 319]